In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
spark = SparkSession.builder.appName("PySparkTables").getOrCreate()

# Define schema
schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("manager_id", IntegerType(), True),
    StructField("salary", IntegerType(), True)
])

# Create DataFrame
data = [
    (3, "Mila", 9, 60301),
    (12, "Antonella", None, 31000),
    (13, "Emery", None, 67084),
    (1, "Kalel", 11, 21241),
    (9, "Mikaela", None, 50937),
    (11, "Joziah", 6, 28485)
]

df = spark.createDataFrame(data, schema=schema)

# Create a temporary view for SQL queries
df.createOrReplaceTempView("Employees")


print("Employees table and view created successfully.")


Employees table and view created successfully.


In [ ]:
Find the IDs of the employees whose salary is strictly less than $30000 and 
whose manager left the company.

In [14]:
spark.sql("""
    SELECT employee_id
    FROM Employees
    WHERE salary < 30000
    AND manager_id IS NOT NULL
    AND manager_id NOT IN (SELECT employee_id FROM Employees)
    ORDER BY employee_id;
""").show()

+-----------+
|employee_id|
+-----------+
|          1|
|         11|
+-----------+



In [15]:
spark.sql("""select manager_id from Employees where manager_id NOT IN (SELECT employee_id FROM Employees)""").show()

+----------+
|manager_id|
+----------+
|         6|
+----------+



In [33]:
from pyspark.sql.functions import *
df_filtered = df.filter(col('salary') < 30000). \
                    filter(col('manager_id').isNotNull())

In [36]:
result_df = df_filtered.alias("f").join(
    df.alias("e"), 
    col("f.manager_id") == col("e.employee_id"), 
    "left_anti"
)

In [37]:
result_df.show()

+-----------+------+----------+------+
|employee_id|  name|manager_id|salary|
+-----------+------+----------+------+
|         11|Joziah|         6| 28485|
+-----------+------+----------+------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 55106)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/spark/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/spark/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/spark/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  File "/spark/python/pyspark/serializers.py", line 595, in read_i